## Add extension to example YAML resources

- read file from folder
- convert yaml to dict
- add element
    with data from file name
    optionally user input - future
- save file back to folder

In [1]:
from pathlib import Path
from json import dumps, loads
from requests import post
from yaml import load as y_load, dump as y_dump
from IPython.display import display, Markdown, HTML

In [38]:
#in_path = Path(r'/Users/ehaas/Documents/FHIR/Davinci-CDEX/input/includes-yaml')
in_path = Path(r'/Users/ehaas/Documents/FHIR/Davinci-CDEX/input/examples-yaml')

fhir_test_server = 'http://test.fhir.org/r4'
fhir_test_server = 'http://hapi.fhir.org/baseR4'
#fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1' 

headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
     }
params = {}

new_pou  = '''
type:
    coding:
    - system: http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp
      code: purpose-of-use
valueCodeableConcept:
    coding:
    - system: http://terminology.hl7.org/CodeSystem/v3-ActReason
      code: COVERAGE
'''

# print(new_pou)

# y_load(new_pou)

new_tag  = '''
system: 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'
code: claims-processing
'''
def convert(r):
    r = post(f'{fhir_test_server}/$convert', params = params, headers = headers, data = r)
    print(r.status_code)
    # view  output
    # return (r.json()["text"]["div"])
    return r.json()

def validate(r): #as dict
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    display(Markdown(f'### {r.status_code} {r.reason}\n{r.json()["text"]["div"]}')) 
    return r.json()

def write(data,f): #as dict 
    display(Markdown(f'### Writing to {f}....')) 
    #out= Path()/ "test_files" / "test.yml"
    out = Path(f)
    out.write_text(y_dump(data, sort_keys=False)) #as yaml

def entitle(name):
    x = name.split('-')
    return(' '.join(i.title() for i in x if i != "example"))

def enname(name):
    x = name.split('-')
    return(' '.join(i for i in x if i != "example"))

def p_name(f):
    #p = loads(f.open().read())
    p=f
    try:
        profile_url = (p['meta']['profile'][0])
        return profile_url.split('/')[-1]
    except KeyError:
        return p['resourceType']

files = [x for x in Path(in_path).iterdir() if x.is_file() and x.name.endswith('yml')]
#files = [x for x in Path(in_path).iterdir() if x.is_file() and x.name.endswith('xml')] # Get xml and convert and rename to json
for f in files:

    data = y_load(f.open().read()) #dict
#     print(data['resourceType'])
    try:
        assert data['resourceType']=="Task"
    except Exception as e:
        pass
    else:
        print(f, data['resourceType'])
        data['meta']['tag']=[y_load(new_tag)]  # add work queue
#         try:  remove reasoncode and add pou
#            print(f"********{data['reasonCode']['text']}*********")
#            assert data['reasonCode']['text'] == 'Support performing requested service'
#            data.pop("reasonCode")
#            data['input'].append(y_load(new_pou))
#         except KeyError:
#            try: 
#               print(data['reasonCode']['text'])
#            except KeyError: 
#               print('no reasonCode')
        data['input'][1]['type']['coding'][0]['system']='http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'  # correct system url
        #print(y_dump(data, sort_keys=False))
        validate(data)
        write(data,f)
        
 

/Users/ehaas/Documents/Python/Venv/venv37/lib/python3.7/site-packages/ipykernel_launcher.py:72: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
/Users/ehaas/Documents/Python/Venv/venv37/lib/python3.7/site-packages/ipykernel_launcher.py:80: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


/Users/ehaas/Documents/FHIR/Davinci-CDEX/input/examples-yaml/cdex-example1-free-text-request.yml Task
no reasonCode


### 412 Precondition Failed
<div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation Outcome</h1><table border="0"><tr><td style="font-weight: bold;">WARNING</td><td>[Task, Line 1, Col 2]</td><td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.meta.tag[0], Line 1, Col 473]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.code.coding[0], Line 1, Col 638]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Task.input[0].type, Line 1, Col 1189]</td>
				<td><pre>Binding for path Task.input[0].type has no source, so can't be checked</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.input[0].type.coding[0], Line 1, Col 1190]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Task.input[1].type, Line 1, Col 1383]</td>
				<td><pre>Binding for path Task.input[1].type has no source, so can't be checked</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.input[1].type.coding[0], Line 1, Col 1384]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.meta.profile[0], Line 1, Col 2]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-data-request' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
		</table>
	</div>

### Writing to /Users/ehaas/Documents/FHIR/Davinci-CDEX/input/examples-yaml/cdex-example1-free-text-request.yml....

/Users/ehaas/Documents/FHIR/Davinci-CDEX/input/examples-yaml/cdex-example3-query-request.yml Task
no reasonCode


...

### 412 Precondition Failed
<div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation Outcome</h1><table border="0"><tr><td style="font-weight: bold;">WARNING</td><td>[Task, Line 1, Col 2]</td><td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.meta.tag[0], Line 1, Col 479]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Task.contained[0].ofType(Bundle).entry[0].resource.ofType(DocumentReference), Line 1, Col 882]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.contained[0].ofType(Bundle).entry[0].resource.ofType(DocumentReference).type.coding[0], Line 1, Col 894]</td>
				<td><pre>Unable to validate code http://loinc.org#34117-2 - Code can not be found in CodeSystem for 'http://loinc.org#34117-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Task.contained[0].ofType(Bundle).entry[1].resource.ofType(DocumentReference), Line 1, Col 114892]</td>
				<td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.contained[0].ofType(Bundle).entry[1].resource.ofType(DocumentReference).type.coding[0], Line 1, Col 114904]</td>
				<td><pre>Unable to validate code http://loinc.org#34117-2 - Code can not be found in CodeSystem for 'http://loinc.org#34117-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">WARNING</td>
				<td>[Task.contained[0].ofType(Bundle), Line 1, Col 596]</td>
				<td><pre>SearchSet Bundles should have a self link that specifies what the search was</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.code.coding[0], Line 1, Col 273792]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Task.input[0].type, Line 1, Col 274259]</td>
				<td><pre>Binding for path Task.input[0].type has no source, so can't be checked</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.input[0].type.coding[0], Line 1, Col 274260]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.input[0].value.ofType(CodeableConcept).coding[0], Line 1, Col 274391]</td>
				<td><pre>Unable to validate code http://loinc.org#34117-2 - Code can not be found in CodeSystem for 'http://loinc.org#34117-2'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Task.input[1].type, Line 1, Col 274503]</td>
				<td><pre>Binding for path Task.input[1].type has no source, so can't be checked</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.input[1].type.coding[0], Line 1, Col 274504]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Task.output[0].type, Line 1, Col 274756]</td>
				<td><pre>Binding for path Task.output[0].type has no source, so can't be checked</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.output[0].type.coding[0], Line 1, Col 274757]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.meta.profile[0], Line 1, Col 2]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-data-request' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
		</table>
	</div>

### Writing to /Users/ehaas/Documents/FHIR/Davinci-CDEX/input/examples-yaml/cdex-example3-query-completed.yml....

/Users/ehaas/Documents/FHIR/Davinci-CDEX/input/examples-yaml/cdex-example1-free-text-completed.yml Task
no reasonCode


### 412 Precondition Failed
<div xmlns="http://www.w3.org/1999/xhtml"><h1>Operation Outcome</h1><table border="0"><tr><td style="font-weight: bold;">WARNING</td><td>[Task, Line 1, Col 2]</td><td><pre>dom-6: 'A resource should have narrative for robust management' Rule 'A resource should have narrative for robust management' Failed</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.meta.tag[0], Line 1, Col 592]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Task.businessStatus, Line 1, Col 775]</td>
				<td><pre>Binding for path Task.businessStatus has no source, so can't be checked</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.code.coding[0], Line 1, Col 817]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Task.input[0].type, Line 1, Col 1368]</td>
				<td><pre>Binding for path Task.input[0].type has no source, so can't be checked</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.input[0].type.coding[0], Line 1, Col 1369]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Task.input[1].type, Line 1, Col 1562]</td>
				<td><pre>Binding for path Task.input[1].type has no source, so can't be checked</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.input[1].type.coding[0], Line 1, Col 1563]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-cdex/CodeSystem/cdex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">INFORMATION</td>
				<td>[Task.output[0].type, Line 1, Col 1820]</td>
				<td><pre>Binding for path Task.output[0].type has no source, so can't be checked</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.output[0].type.coding[0], Line 1, Col 1821]</td>
				<td><pre>Unknown Code System 'http://hl7.org/fhir/us/davinci-hrex/CodeSystem/hrex-temp'</pre></td>
			</tr>
			<tr>
				<td style="font-weight: bold;">ERROR</td>
				<td>[Task.meta.profile[0], Line 1, Col 2]</td>
				<td><pre>Profile reference 'http://hl7.org/fhir/us/davinci-cdex/StructureDefinition/cdex-task-data-request' has not been checked because it is unknown, and the validator is set to not fetch unknown profiles</pre></td>
			</tr>
		</table>
	</div>

### Writing to /Users/ehaas/Documents/FHIR/Davinci-CDEX/input/examples-yaml/cdex-example1-free-text-completed.yml....